<a href="https://colab.research.google.com/github/connorfarrell7/Predicting_Placements_for_Engineering_Students/blob/mmetwalli96/Student_Placement_Records_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing dependencies
import psycopg2
import getpass
import pandas as pd

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
# Connect to the database
server_pass = getpass.getpass('Enter database password: ')
engine = psycopg2.connect(
    database="postgres",
    user="group_4",
    password=server_pass,
    host="capstone-project.c9uk6piofxnr.us-east-2.rds.amazonaws.com",
    port='5432'
)

Enter database password: ··········


In [3]:
# Importing the data from the databse 
sql = "select * from student_data"
student_records_df = pd.read_sql_query(sql, engine)
student_records_df.head()

,age,gender,stream,internships,cgpa,hostel,history_of_backlogs,placed_or_not,id
0,22,Male,Electronics And Communication,1,8,1,1,1,1
1,21,Female,Computer Science,0,7,1,1,1,2
2,22,Female,Information Technology,1,6,0,0,1,3
3,21,Male,Information Technology,0,8,0,1,1,4
4,22,Male,Mechanical,0,8,1,0,1,5


In [ ]:
# Creating a pandas series to know the age groups and the number of students in each one and how many were placed or not

# Grouping based on age
age_groups = student_records_df['age'].value_counts()
age_groups = pd.DataFrame({'Total Count' : age_groups.values.tolist()}, index = age_groups.index.to_list())
age_groups.index.name='Age'

# Splitting the student records to placed and not placed
placed_students = student_records_df[student_records_df['placed_or_not'] == 1]
not_placed_students = student_records_df[student_records_df['placed_or_not'] == 0]

# Placed students grouped by age
age_groups_placed = placed_students['age'].value_counts()
age_groups_placed = pd.DataFrame({'Placed Students Count' : age_groups_placed.values.tolist()}, index = age_groups_placed.index.to_list())
age_groups_placed.index.name = 'Age'

# Not placed students grouped by age
age_groups_not_placed = not_placed_students['age'].value_counts()
age_groups_not_placed = pd.DataFrame({'Non-Placed Students Count' : age_groups_not_placed.values.tolist()}, index = age_groups_not_placed.index.to_list())
age_groups_not_placed.index.name = 'Age'

# Grouped student records by age and placement
age_groups_df = pd.concat([age_groups_placed, age_groups_not_placed, age_groups], axis=1)
age_groups_df = age_groups_df.fillna(0).astype({'Non-Placed Students Count': 'int64'})
age_groups_df

,Placed Students Count,Non-Placed Students Count,Total Count
Age,,,
19,92,64,156
20,199,176,375
21,617,467,1084
22,478,463,941
23,85,110,195
24,104,27,131
25,22,7,29
26,37,13,50
28,3,0,3


In [ ]:
# Creating a pandas dataframe to get the average of age and CGPA by stream
avg_by_stream = student_records_df.groupby('stream').mean().drop(['internships', 'hostel', 'history_of_backlogs', 'placed_or_not', 'id'], 1)
avg_by_stream = avg_by_stream.rename(columns={"age": "Age", "cgpa": "CGPA"})
avg_by_stream 

,Age,CGPA
stream,,
Civil,21.441640,7.094637
Computer Science,21.559278,7.039948
Electrical,21.299401,7.080838
Electronics And Communication,21.410377,7.125000
Information Technology,21.539797,7.073806
Mechanical,21.518868,7.063679


In [ ]:
# Group student placement by gender

# Total students
gender = student_records_df['gender'].value_counts()
gender = pd.DataFrame({'Total Count': gender.values.tolist()}, index= gender.index.to_list())
gender.index.name = 'Gender'

# Placed studnets
gender_placed  = placed_students['gender'].value_counts()
gender_placed = pd.DataFrame({'Placed Students Count': gender_placed.values.tolist()}, index= gender_placed.index.to_list())

# Non-Placed students
gender_not_placed = not_placed_students['gender'].value_counts()
gender_not_placed = pd.DataFrame({'Non-Placed Students Count': gender_not_placed.values.tolist()}, index= gender_not_placed.index.to_list())

# Average CGPA and age by gender
avg_by_gender = student_records_df.groupby('gender').mean().drop(['internships', 'hostel', 'history_of_backlogs', 'placed_or_not', 'id'], 1)
avg_by_gender = avg_by_gender.rename(columns={"age": "Age", "cgpa": "CGPA"})

# Grouped student records by gender and placement
gender_df = pd.concat([gender_placed, gender_not_placed, gender, avg_by_gender], axis=1)
gender_df

,Placed Students Count,Non-Placed Students Count,Total Count,Age,CGPA
Male,1364,1111,2475,21.473131,7.072323
Female,275,216,491,21.549898,7.081466


In [ ]:
x = student_records_df[['gender', 'stream', 'age']].value_counts()

In [ ]:
sql = 'select count(age) from student_data group by age, gender'
df = pd.read_sql_query(sql, engine)

In [ ]:
df

,count
0,166
1,1
2,185
3,899
4,10
5,25
6,772
7,1
8,113
9,138
